# Automatic speech recognition(ASR) / STT
## Whisper - OpenAI

### Speech To Text package

In [1]:
import session_info

session_info.show()

In [2]:
import os
os.listdir("./data")

['test_data1.mp3',
 'test_data2.mp3',
 'test_data3.mp3',
 'test_data4.mp3',
 'test_data5.mp3']

In [3]:
import whisper
import numpy as np
import ffmpeg
import torch

In [4]:
model = whisper.load_model("base")

## Case 1. 단문

* 원래 문장: 마음으로 일출을 기다렸다.

In [8]:
# case 1. 단문
result1 = model.transcribe("./data/test_data1.mp3", fp16 = False)
print(result1["text"])

 마음으로 일출을 기다렸다.


## Case 2. 중간에 끊기는 문장

* 원래 문장: 여행의 첫날을 주변을 한번 둘러보는 탐색의 날로 정한 우리는 해가 늦게 뜨는 이곳에 겨울의 특성 덕분에 충분한 수면을 취한 뒤잠에서 깨어날 수 있었다. 우리는 (--잠시 끊김--) 아침 식사를 배부르게 즐긴 뒤 /

In [9]:
# case 2. 중간에 끊기는 음원
result2 = model.transcribe("./data/test_data2.mp3", fp16 = False)
print(result2["text"])

 여행에 첫날을 주변을 한번 둘러보는 탐색에 날로 중한 우리는 해가 늦게 들은 이곳의 겨울에 특성도 구네 충분하우스맨을 쉬어 안 뒤 잠에서 깨어날 수 있었다. 우리는 아침 식사를 배부르게 즐긴 뒤 Kelley Mod Infinite


## Case 3.  효과음이 섞인 음원

* 원래 문장: 우리는 가지고 있는 것 중에서도 가장 따뜻한 겨울 옷을 찾아 입었다. 따뜻한 방한 슈트에 털 부츠와 장갑 모자를 더하고 목도리를 두르는 것도 잊지 않았다. (스석이는 소리 삽입됨)

## Case 4.  의문문이 들어간 음원

* 원래 문장: 얼굴에는 콜드 크림을 발랐다. 온기가 빠져나가지 않도록 피부에 막을 씌워주어 저체온증에 빠지는 것으로부터 나를 보호해줄 것이다. 단단히 무장한 덕분이었을까? 나는 몸을 따뜻하게 유지한 채로 내 뺨에 닿는 북극의 차가운 공기를 즐기고 있다. 

In [11]:
# case 4. 의문문이 들어간 음원
result4 = model.transcribe("./data/test_data4.mp3", fp16 = False)
print(result4["text"])

 얼굴에는 콜드 크림을 발랐다. 옮기가 빠져나가지 않도록 피부에 막을 씌워주어 저체운 중에 빠지는 것으로부터 나를 보호해줄 것이다. 단단히 무장한 높은이었을까? 나는 봉을 따뜻하게 유지한 채로 내 뼈에 닿는 복극의 차가운 공비를 즐기고 있다.


In [18]:
# low level access

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("./data/test_data4.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key = probs.get)}")

# decode the audio
options = whisper.DecodingOptions(fp16 = False)
result_low_lv = whisper.decode(model, mel, options)

# print the recognized text
print(result_low_lv.text)

Detected language: ko
얼굴에는 콜드 크림을 발랐다. 옮기가 빠져나가지 않도록 피부에 막을 씌워주어 저체운 중에 빠지는 것으로부터 나를 보호해줄 것이다. 단단히 무장한 높은이었을까? 나는 봉을 따뜻하게 유지한 채로 내 뼈에 닿는 복극의 차가운


## Case 5. 30초 이상의 음원

* 원래 문장: 전날 저녁 라플란드에 도착한 우리는 극권의 북쪽에 위치한 작은 마을에서 첫 번째 밤을 보냈다. 벽난로와 고풍스러운 카펫 쿠션으로 꾸며진 안락한 나무 오두막이 우리의 숙소였다. 여행의 첫날을 주변을 한번 둘러보는 탐색의 날로 정한 우리는 해가 늦게 뜨는 이곳에 겨울의 특성 덕분에 충분한 수면을 취한 뒤 잠에서 깨어날 수 있었다. 우리는 아침 식사를 배부르게 즐긴 뒤 느긋한 마음으로 일출을 기다렸다. 우리는 가지고 있는 것 중에서도 가장 따뜻한 겨울옷을 찾아 입었다. 따뜻한 방한 슈트에 털부츠와 장갑 모자를 더하고 목도리를 두르는 것도 잊지 않았다. 신체 부위 중 차가운 곳은 공기가 닿는 얼굴이 유일하다. 얼굴에는 콜드 크림을 발랐다. 온기가 빠져나가지 않도록 피부에 막을 씌워주어 저체온증에 빠지는 것으로부터 나를 보호해줄 것이다. 단단히 무장한 덕분이었을까? 나는 몸을 따뜻하게 유지한 채로 내 뺨에 닿는 북극의 차가운 공기를 즐기고 있다. 이곳은 몇 년 전 리아가 여행을 왔던 장소이기도 하다. 당시의 경험을 통해 이 마을을 잘 알게 된 리아는 이 근처에 나에게 꼭 소개하고 싶은 카페가 있다고 말했다. 그 카페에 가기 위해 우리는 하이킹 지팡이를 들고 길을 나선다. 사람들의 발길이 닿아 어느새 눈길은 평평하게 다져진 상태이다. 

In [20]:
result5  = model.transcribe("./data/test_data5.mp3", fp16 = False)
print(result5["text"])

 전날 저녁 라플라인데 도착한 우리는 그건의 북쪽에 위치한 작은 마을에서 첫 번째 밤을 보냈다. 병날로와 고풍스러운 카페. 쿠션으로 꾸며진 알락한 나무 오동하게 우리의 숙소였다. 여행의 첫나를 주변을 한 번 둘러보는 탐색의 날로 중한 우리는 해가 늦게들은 이것의 겨울에 특성도 구네 충분한 수맨을 취한 뒤 잠에서 깨어날 수 있었다. 우리는 아침 식사를 배부르게 즐긴 뒤, 느그탄 마음으로 일출을 기다렸다. 우리는 가지고 있는 것 중에서도 가장 따뜻한 겨울 옷을 찾아 입었다. 따뜻한 방한 슈트에 털 부추와 장갑 모자를 더하고 목도리를 들은 것도 잊지 않았다. 신체 보의 중 차가운 곳은 공기가 나는 얼굴이 유일하다. 얼굴에는 콜드크림을 발랐다. 옮기가 빠져나가지 않도록 피부에 막을 씌워 주어 저체 온 중에 빠지는 것으로부터 나를 보호해 줄 것이다. 단단히 무장한 높은이었을까? 나는 봉을 따뜻하게 유지한 채로 내 뼈에 닿는 북극의 차가운 공기를 즐기고 있다. 이것은 몇 년 전 리아가 여행을 왔던 정소에기도 하다. 당시의 경험을 통해 이 마을을 잘 알게 된 리아는 이건 저의 나에게 꼭 소개하고 싶은 카페가 있다고 말했다. 그 카페의 가기 위에 우리는 하이킹 집 방위를 들고 길을 나선다. 쓰란들의 발길이다 어느새 눈길은 평평하게 다져진 상태다.


전체 파일을 30초 단위의 sliding window를 통해 변환하므로 무리없이 잘 작동함

Internally, the `transcribe()` method reads the entire file and processes the audio with a sliding 30-second window, performing autoregressive sequence-to-sequence predictions on each window.

## Compare vs speech_recognition

In [25]:
# convert from mp3 to wav

from os import path
from pydub import AudioSegment

# files
src = "./data/test_data4.mp3"
dst = "./data/test_data4.wav"

sound = AudioSegment.from_mp3(src)
sound.export(dst, format = "wav")

<_io.BufferedRandom name='./data/test_data4.wav'>

In [22]:
import speech_recognition as sr
sr.__version__

'3.8.1'

In [23]:
r = sr.Recognizer()

Each Recognizer instance has seven methods for recognizing speech from an audio source using various APIs. These are:

- **`recognize_bing()`**: Microsoft Bing Speech
- **`recognize_google()`**: Google Web Speech API
- **`recognize_google_cloud()`**: Google Cloud Speech - requires installation of the google-cloud-speech package
- **`recognize_houndify()`**: Houndify by SoundHound
- **`recognize_ibm()`**: IBM Speech to Text
- **`recognize_sphinx()`**: CMU Sphinx - requires installing PocketSphinx
- **`recognize_wit()`**: Wit.ai

In [27]:
test = sr.AudioFile("./data/test_data4.wav")
with test as source:
    result_sr = r.record(source)
r.recognize_google(result_sr, language = 'ko-KR')

'얼굴에는 콜드크림을 발랐다 1기가 빠져 나가지 않도록 피부에 막을 심어주어야 저체온증에 빠지는 것으로부터 나를 보호해 줄 것이다 단단히 무장한 덕분이었을까 나는 몸을 따뜻하게 유지한 채로 내 마음에 닿는 그게 차가운 공기를 즐기고 있다'